In [ ]:
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.factors import SimpleMovingAverage,RSI
from quantopian.research import run_pipeline
from quantopian.pipeline import Pipeline,CustomFilter
from quantopian.pipeline.factors import CustomFactor,RSI
from quantopian.pipeline.data import morningstar
from zipline import TradingAlgorithm  
from quantopian.pipeline.filters import Q1500US, Q500US
from quantopian.pipeline.factors import AverageDollarVolume
from quantopian.pipeline.classifiers.morningstar import Sector, SuperSector
#
#from quantopian.pipeline.data.alpha_vertex import precog_top_500 as precog
#
import numpy as np
import talib 
import pandas as pd 
import matplotlib.pyplot as plt 
import matplotlib.cm as cm
from datetime import timedelta, datetime

# For use in your algorithms
# Using the full dataset in your pipeline algo
from quantopian.pipeline.data.eventvestor import EarningsCalendar

# To use built-in Pipeline factors for this dataset
from quantopian.pipeline.factors.eventvestor import (
BusinessDaysUntilNextEarnings,
BusinessDaysSincePreviousEarnings
)
   

In [ ]:
def make_pipeline():
    close = USEquityPricing.close
    open_ = USEquityPricing.open
    high = USEquityPricing.high
    low = USEquityPricing.low
    
    pipe = Pipeline(
        columns = { 'close': close.latest,}, 
        screen = Q1500US())
    return pipe 


df_surprise = local_csv("surprise.csv", symbol_column="Symbol", date_column=':Time', use_date_column_as_index=True,timezone='UTC' )
df_surprise = df_surprise.ix[:"2017-7-1"] # 後ろにバッファをもたせる
df_surprise = df_surprise[df_surprise[":%Suprise"] != 0]
df_surprise = df_surprise.dropna()
dates = df_surprise.sort_index().index.unique()
spy = get_pricing("SPY", start_date=dates[0], end_date= dates[-1].to_datetime() + timedelta(days=10), frequency='daily', fields='price')
spy = spy.pct_change()
market_dates = spy.index
pipeline_results = run_pipeline(make_pipeline(), start_date=dates[0], end_date=dates[-1].to_datetime() + timedelta(days=10))


In [ ]:
df_surprise.sort_index().tail()

In [ ]:

results = list()
month = 0
for i, d in enumerate(market_dates):
    if d.strftime("%m") != month:
        print d.strftime("%Y-%m-%d") 
        month = d.strftime("%m")
    if d in dates:
        try:
            ## US1500 に入っている銘柄だけ取得
            syms = df_surprise.ix[d]["Symbol"].apply(lambda x: x in pipeline_results.ix[d].index)
            surprise = df_surprise.ix[d][syms]
            surprise = surprise[["Symbol",":%Suprise"]].reset_index(drop=True).set_index("Symbol")
            data = get_pricing(surprise.index, start_date=d ,end_date=market_dates[i+5],fields="price")
            data = data.pct_change().sub(spy.ix[data.index], axis=0)
            data = data.reset_index(drop=True).T

            results.append(pd.concat([surprise, data], axis=1,))
            
        except:
            print "Error Occurs on %s" % d.strftime("%Y-%m-%d")
            #print syms 
            

df = pd.concat(results)
df[0]=0
df = df.dropna()



In [ ]:
df.tail()

In [ ]:
x = df[((df[1] > 0.2) | (df[1] < -0.2))] # & (df[":%Suprise"] < 0) 
#x = df.drop(":%Suprise", axis=1)
x.drop(":%Suprise", axis=1).T.plot(legend=False)
x.drop(":%Suprise", axis=1).T.cumsum().plot(legend=False)
import matplotlib.cm as cm
fig = plt.figure()
im = plt.scatter(x[1], x[2],
 c=x[":%Suprise"],
 linewidths=0, alpha=1, 
 cmap=cm.copper # ここでカラーマップを指定
 )
fig.colorbar(im)

#x.plot(kind='scatter', x=1, y=2)



In [ ]:
x.sort_values(by=":%Suprise")

In [ ]:
df[df[1] < -0.25].drop(":%Suprise", axis=1).T.plot(legend=False)
len(df)

#~~earnings_calendar使用~~ USEquityPricingでプライスを取るとアジャストされていないデータということがわかったので，これは使わない

In [ ]:

# Import necessary Pipeline modules
from quantopian.pipeline import Pipeline
from quantopian.research import run_pipeline
from quantopian.pipeline.factors import AverageDollarVolume
from quantopian.pipeline.factors import CustomFactor
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.filters import Q1500US, Q500US

# For use in your algorithms
# Using the full dataset in your pipeline algo
from quantopian.pipeline.data.eventvestor import EarningsCalendar

# To use built-in Pipeline factors for this dataset
from quantopian.pipeline.factors.eventvestor import (
BusinessDaysUntilNextEarnings,
BusinessDaysSincePreviousEarnings
)
   
def make_pipeline():
    close = BusinessDaysSincePreviousEarnings()
    
    pipe = Pipeline(
        columns = { 'pe': pe, }, 
        screen = Q500US()
                   )
    return pipe 

results = run_pipeline(make_pipeline(), start_date='2007-01-01', end_date='2015-07-19')


In [ ]:
df = results[results["pe"] == 0]
#dates = df.index.get_level_values(0).unique()
spy = get_pricing("SPY", start_date=dates[0], 
                  end_date= dates[-1].to_datetime() + timedelta(days=10), 
                  frequency='daily', fields='price')

spy = spy.pct_change()
dates = spy.index
l = list()
for i, date in enumerate( dates[:30]):
    try:
        syms = df.ix[date].index 
        data = get_pricing(
            syms, start_date=date, end_date=dates[i+5], frequency='daily', fields='price')
        data = data.pct_change().sub(spy.ix[data.index], axis=0)
        data = data.reset_index(drop=True).T
        l.append(data)
            
    except:
        print date.strftime("%Y-%m-%d")
            
        
    
    

In [ ]:
x = pd.concat(l)
x[0] = 0

x[(x[1]<-0.02)].plot(kind='scatter', x=1, y = 2) #(x[1]>0.02) | 
x.plot(kind='scatter', x=1, y = 2)

In [ ]:
x.describe()


In [ ]:
x[['return','return2','return3','return4']].T.plot(legend=False)
x.plot(kind='scatter', x='return', y='return2')

In [ ]:
x = df.copy()
all_dates = x.index.get_level_values(0).unique()

spy = get_pricing("spy", start_date=all_dates[0], end_date=all_dates[-1], frequency='daily', fields='close_price')
spy = spy.pct_change()

l = list()
for date in x.index.get_level_values(0).unique():
    try:
        df = x.ix[date]
        data = get_pricing(df.index.tolist(),
                       start_date=date, 
                       end_date=all_dates[all_dates.get_loc(date) + 3],
                       frequency = 'daily',
                       fields='close_price')
        change = data.pct_change()
        change = change.subtract(spy[data.index], axis=0)
        df["return2"] = change.ix[1]
        
    except:
        pass
    l.append(df) #.reset_index(drop=True)
    



In [ ]:
df = results[results["pe"] == 0]
df